In [82]:
import numpy as np
import math

In [83]:
imgs = np.random.rand(1,1,20,20)
header = "{},{},{},{}".format(*imgs.shape)

In [84]:
channels, X, Y = imgs[0].shape

In [85]:
class conv2d:
    def __init__(self, k, s, p):
        self.k = k
        self.p = p
        self.s = s

    def get_x(self):
        return self.X + self.p * 2

    def get_y(self):
        return self.Y + self.p * 2

    def get_win_val(self, w, i, j, ch, imgs, img):    
        h_slides = math.floor((self.get_x()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_y()-self.k)/self.s) + 1

        r = math.floor(w / h_slides)
        c = w % h_slides
        x = (r * self.s) + i - self.p
        y = (c * self.s) + j - self.p

        if x < 0 or y < 0 or x >= self.X or y >= self.Y:
            return 0

        return imgs[img][ch][x][y]
    
    def img_to_conv2d(self, imgs):
        self.channels, self.X, self.Y = imgs[0].shape

        h_slides = math.floor((self.get_x()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_y()-self.k)/self.s) + 1

        windows = h_slides * v_slides

        data = []

        for w in range(windows):
            row = []
            for c in range(channels):
                for i in range(self.k):
                    for j in range(self.k):
                        row.append(self.get_win_val(w, i, j, c, imgs, 0))
            data.append(row)

        return np.array(data)

In [86]:
p = 0

def get_x():
    return X + p * 2

def get_y():
    return Y + p * 2

def get_win_val(w, i, j, s, ch, k, img):    
    h_slides = math.floor((get_x()-k)/s) + 1
    v_slides = math.floor((get_y()-k)/s) + 1
    
    r = math.floor(w / h_slides)
    c = w % h_slides
    x = (r * s) + i - p
    y = (c * s) + j - p
    
    if x < 0 or y < 0 or x >= X or y >= Y:
        return 0
    
    return imgs[img][ch][x][y]

def img_to_conv2d():
    s = 2
    k = 4

    h_slides = math.floor((get_x()-k)/s) + 1
    v_slides = math.floor((get_y()-k)/s) + 1
    
    windows = h_slides * v_slides
    
    data = []
    
    for w in range(windows):
        row = []
        for c in range(channels):
            for i in range(k):
                for j in range(k):
                    row.append(get_win_val(w, i, j, s, c, k, 0))
        data.append(row)
    
    return np.array(data)

In [87]:
con = conv2d(4, 2, 0)
data = con.img_to_conv2d(imgs)

data1 = img_to_conv2d()
np.array_equal(data, data1)

True

In [88]:
data.shape

(81, 16)

In [7]:
imgs[0][0].shape

(20, 20)

In [8]:
def split(array, nrows, ncols):
    """Split a matrix into sub-matrices."""
    r, h = array.shape
    if r % nrows != 0:
        padding = (math.ceil(r / nrows) * nrows) - r
        array = np.vstack((array, np.zeros((padding, h))))
        r, h = array.shape
    if h % ncols != 0:
        padding = (math.ceil(h / ncols) * ncols) - h
        array = np.hstack((array, np.zeros((r, padding))))
        r, h = array.shape
    num_x_blocks = math.ceil(r / float(nrows))
    num_y_blocks = math.ceil(h / float(ncols))

    rows = np.vsplit(array, num_x_blocks)
    return [np.array(np.hsplit(row, num_y_blocks)) for row in rows]

In [17]:
splits = split(imgs[0][0], 10, 10)
splits = np.concatenate(np.array(splits))

In [63]:
# Vertical splits
a = splits[0][:,-2:]
b = splits[1][:,:2]
c = splits[2][:,-2:]
d = splits[3][:,:2]

# horizontal splits
e = splits[0][-2:,:]
f = splits[2][:2,:]
g = splits[1][-2:,:]
h = splits[3][:2,:]

# corner splits
i = splits[0][-2:,-2:]
j = splits[1][-2:,:2]
k = splits[2][:2,-2:]
l = splits[3][:2,:2]

ab = np.hstack((a,b))
cd = np.hstack((c,d))

ef = np.vstack((e,f))
gh = np.vstack((g,h))

ijkl = np.hstack(( np.vstack((i, j)), np.vstack((k, l)) ))

con = conv2d(4, 2, 0)
m = con.img_to_conv2d(splits[0].reshape(1,1,10,10))
con = conv2d(4, 2, 0)
n = con.img_to_conv2d(ab.reshape(1,1,10,4))
con = conv2d(4, 2, 0)
o = con.img_to_conv2d(splits[1].reshape(1,1,10,10))

con = conv2d(4, 2, 0)
p = con.img_to_conv2d(ef.reshape(1,1,4,10))
con = conv2d(4, 2, 0)
q = con.img_to_conv2d(ijkl.reshape(1,1,4,4))
con = conv2d(4, 2, 0)
r = con.img_to_conv2d(gh.reshape(1,1,4,10))

con = conv2d(4, 2, 0)
s = con.img_to_conv2d(splits[2].reshape(1,1,10,10))
con = conv2d(4, 2, 0)
t = con.img_to_conv2d(cd.reshape(1,1,10,4))
con = conv2d(4, 2, 0)
u = con.img_to_conv2d(splits[3].reshape(1,1,10,10))

In [72]:
n.shape

(4, 16)

In [68]:
16+4+16+4+1+4+16+4+16

81

In [75]:
math.floor((20-4)/2) + 1, math.floor((20-4)/2) + 1

(9, 9)

In [73]:
"""
10x10 - 4x4
10x4  - 4x1
4x4   - 1x1
4x10  - 1x4
"""

m = [m[:4,:], m[4:8,:], m[8:12,:], m[12:16,:]]
n = [n[i] for i in range(4)]
o = [o[:4,:], o[4:8,:], o[8:12,:], o[12:16,:]]

In [80]:
k = np.vstack(( m[0], n[0], o[0] ))
for i in range(k.shape[0]):
    print(k[i].tolist())

[0.8696897234909104, 0.09533342043525395, 0.4643393578391408, 0.907208428924151, 0.6349884606136865, 0.08411247152033707, 0.06833496125602212, 0.8933541376652471, 0.7700031976837484, 0.8169831173740064, 0.7881946561654846, 0.9931801138244228, 0.18633547568525455, 0.8579012261376571, 0.7034103520770748, 0.8369643695327349]
[0.4643393578391408, 0.907208428924151, 0.7452724245351724, 0.3545438785258658, 0.06833496125602212, 0.8933541376652471, 0.7714564783278428, 0.2974963356074217, 0.7881946561654846, 0.9931801138244228, 0.1873335833453572, 0.3408456005846211, 0.7034103520770748, 0.8369643695327349, 0.5581339334774951, 0.022197319466386256]
[0.7452724245351724, 0.3545438785258658, 0.7296589510703292, 0.4181830736022807, 0.7714564783278428, 0.2974963356074217, 0.9636085360616199, 0.5463299505964364, 0.1873335833453572, 0.3408456005846211, 0.806981830085967, 0.840472780040262, 0.5581339334774951, 0.022197319466386256, 0.6935144848111555, 0.9808784430995319]
[0.7296589510703292, 0.418183073

In [78]:
for i in range(data[:9,:].shape[0]):
    print(data[:9,:][i].tolist())

[0.8696897234909104, 0.09533342043525395, 0.4643393578391408, 0.907208428924151, 0.6349884606136865, 0.08411247152033707, 0.06833496125602212, 0.8933541376652471, 0.7700031976837484, 0.8169831173740064, 0.7881946561654846, 0.9931801138244228, 0.18633547568525455, 0.8579012261376571, 0.7034103520770748, 0.8369643695327349]
[0.4643393578391408, 0.907208428924151, 0.7452724245351724, 0.3545438785258658, 0.06833496125602212, 0.8933541376652471, 0.7714564783278428, 0.2974963356074217, 0.7881946561654846, 0.9931801138244228, 0.1873335833453572, 0.3408456005846211, 0.7034103520770748, 0.8369643695327349, 0.5581339334774951, 0.022197319466386256]
[0.7452724245351724, 0.3545438785258658, 0.7296589510703292, 0.4181830736022807, 0.7714564783278428, 0.2974963356074217, 0.9636085360616199, 0.5463299505964364, 0.1873335833453572, 0.3408456005846211, 0.806981830085967, 0.840472780040262, 0.5581339334774951, 0.022197319466386256, 0.6935144848111555, 0.9808784430995319]
[0.7296589510703292, 0.418183073

In [81]:
np.array_equal(k, data[:9,:])

True